In [9]:
import numpy as np
import torch
from regen_pipe import ReSDPipeline
from PIL import Image

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:

class DiffWMAttacker:
    def __init__(self, pipe, noise_step=60, captions={}):
        self.pipe = pipe
        self.noise_step = noise_step
        self.captions = captions
        print(
            f"Diffuse attack initialized with noise step {self.noise_step} and use prompt {len(self.captions)}"
        )

    def attack(self, image, return_latents=False, return_dist=False):
        with torch.no_grad():
            generator = torch.Generator(DEVICE).manual_seed(1024)
            latents_buf = []
            prompts_buf = []
            outs_buf = []
            timestep = torch.tensor(
                [self.noise_step], dtype=torch.long, device=DEVICE
            )
            ret_latents = []

            def batched_attack(latents_buf, prompts_buf, outs_buf):
                latents = torch.cat(latents_buf, dim=0)
                images = self.pipe(
                    prompts_buf,
                    head_start_latents=latents,
                    head_start_step=50 - max(self.noise_step // 20, 1),
                    guidance_scale=7.5,
                    generator=generator,
                )
                images = images[0]
                for img, out in zip(images, outs_buf):
                    return img

            img = np.asarray(image) / 255
            img = (img - 0.5) * 2
            img = torch.tensor(img).permute(2, 0, 1).unsqueeze(0)
            latents = self.pipe.vae.encode(
                img.to(device=DEVICE, dtype=torch.float16) #remove when running on cuda?
            ).latent_dist
            latents = latents.sample(generator) * self.pipe.vae.config.scaling_factor
            noise = torch.randn(
                [1, 4, img.shape[-2] // 8, img.shape[-1] // 8], device=DEVICE
            )

            latents = self.pipe.scheduler.add_noise(latents, noise, timestep).type(
                torch.half
            )
            latents_buf.append(latents)
            outs_buf.append("")
            prompts_buf.append("")

            img = batched_attack(latents_buf, prompts_buf, outs_buf)
            return img


def remove_watermark(attack_method, image, strength, model):
    # create attacker
    print(f"Creating attacker {attack_method}...")
    if attack_method == "regen_diffusion":
        pipe = ReSDPipeline.from_pretrained(
            model, torch_dtype=torch.float16, revision="fp16"
        )
        # pipe.set_progress_bar_config(disable=True)
        pipe.to(DEVICE)
        attacker = DiffWMAttacker(pipe, noise_step=strength, captions={})

    else:
        raise Exception(f"Unknown attacking method: {attack_method}!")

    img = attacker.attack(image)
    return img

def rinse_4xDiff(image, strength, model="CompVis/stable-diffusion-v1-4"):
    first_attack = True
    for attack in [
        "regen_diffusion",
        "regen_diffusion",
        "regen_diffusion",
        "regen_diffusion",
    ]:
        if first_attack:
            image = remove_watermark(attack, image, strength, model)
            first_attack = False
        else:
            image = remove_watermark(attack, image, strength, model)
    return image


In [ ]:
# Run attack on an image, for example
image = Image.open("./watermarked.png")
# image.show()
rinse_4xDiff(image, strength=60)

text_encoder/model.safetensors not found


Creating attacker regen_diffusion...


Loading pipeline components...:  57%|█████▋    | 4/7 [00:00<00:00, 16.00it/s]An error occurred while trying to fetch /Users/tomg/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /Users/tomg/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /Users/tomg/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /Users/tomg/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error in

Diffuse attack initialized with noise step 60 and use prompt 0
